In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)
sys.path.append(parent_directory)
from Class.jenie import Jenie
import pandas as pd
import numpy as np
import scanpy as sc
import anndata
import umap.umap_ as umap
import leidenalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import pairwise_distances
from scipy.stats import zscore
from matplotlib.colorbar import ColorbarBase
import matplotlib.colors as mcolors
from umap import UMAP
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from kneed import KneeLocator
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ParameterGrid

In [3]:
import doctest
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.datasets import make_blobs
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse
from sklearn.neighbors import NearestNeighbors
import scanpy as sc
#from geopy.distance import geodesic
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
import umap.umap_ as umap
#from umap import UMAP
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#from leidenalg import find_partition
#import networkx as nx

# two useful data viz libraries
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.cluster as cluster
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering
from kneed import KneeLocator
from sklearn.model_selection import ParameterGrid
from sklearn.cluster import DBSCAN
#import hdbscan
#from sklearn.cluster import HDBSCAN
# setup plotting in a notebook in a reasonable way
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [4]:
plt.style.use('dark_background')

### Load all data

In [5]:
xenium_scdata = sc.read_h5ad(r"D:\SMI-0214_DonCleveland_UCSD\SMI-0214_DonCleveland_UCSD\Xenium_mouse_filtered.h5ad")
xenium_scdata

AnnData object with n_obs × n_vars = 547139 × 248
    obs: 'cell_id', 'Sample ID', 'Number of genes', 'Number of transcripts', 'leiden', 'cluster_names', 'celltype'
    uns: 'Sample ID_colors', 'cell_id_colors', 'celltype_colors', 'cluster_names_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_spatial_rescaled', 'X_spatial_rescaled_flipped', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [6]:
xenium_scdata.var.index

Index(['2010300C02Rik', 'Acsbg1', 'Acta2', 'Acvrl1', 'Adamts2', 'Adamtsl1',
       'Adgrl4', 'Aldh1a2', 'Angpt1', 'Ano1',
       ...
       'Trp73', 'Trpc4', 'Unc13c', 'Vat1l', 'Vip', 'Vwc2l', 'Wfs1', 'Zfp366',
       'Zfp536', 'Zfpm2'],
      dtype='object', length=248)

In [7]:
cosmx_scdata = sc.read_h5ad(r"D:\SMI-0214_DonCleveland_UCSD\SMI-0214_DonCleveland_UCSD\Cosmx_mouse_filtered.h5ad")
cosmx_scdata

AnnData object with n_obs × n_vars = 132721 × 950
    obs: 'total_counts', 'leiden', 'celltype'
    var: 'mean', 'std'
    uns: 'celltype_colors', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [8]:
gellyfish_scdata = sc.read_h5ad(r"D:\SMI-0214_DonCleveland_UCSD\SMI-0214_DonCleveland_UCSD\Gellyfish Analysis\CBM5_new_PTB.h5ad")
gellyfish_scdata

AnnData object with n_obs × n_vars = 152568 × 223
    obs: 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'leiden', 'leiden_denovo', 'Experiment', 'Age', 'Section', 'Mouse', 'Treatment', 'Timepoint', 'Ptbp1', 'MERFISH celltype', 'New_PTB'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'Mouse_colors', 'Section_colors', 'Treatment_colors', 'leiden', 'leiden_colors', 'leiden_denovo_colors', 'log1p', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_spatial', 'X_umap', 'X_umap_denovo', 'rep'
    varm: 'PCs'
    layers: 'Raw counts'
    obsp: 'connectivities', 'distances'

### Create overlap gene list

In [9]:
var_names = xenium_scdata.var_names.intersection(cosmx_scdata.var_names).intersection(gellyfish_scdata.var_names)
var_names

Index(['Acta2', 'Aqp4', 'Bdnf', 'Calb1', 'Cldn5', 'Fos', 'Gad1', 'Gfap',
       'Gpr17', 'Npy2r', 'Pdgfra', 'Slit2', 'Vip'],
      dtype='object')

### Xenium preprocessing

In [10]:
xenium_overlap_scdata = xenium_scdata
xenium_overlap_scdata.X = xenium_overlap_scdata.obsm['X_raw']
xenium_overlap_scdata = xenium_scdata[:, var_names]
xenium_overlap_scdata

View of AnnData object with n_obs × n_vars = 547139 × 13
    obs: 'cell_id', 'Sample ID', 'Number of genes', 'Number of transcripts', 'leiden', 'cluster_names', 'celltype'
    uns: 'Sample ID_colors', 'cell_id_colors', 'celltype_colors', 'cluster_names_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_spatial_rescaled', 'X_spatial_rescaled_flipped', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [11]:
xenium_scdata.obs

,cell_id,Sample ID,Number of genes,Number of transcripts,leiden,cluster_names,celltype
1,2,C3m1,12.0,12,12,Micro-PVM,Microglia
2,3,C3m1,34.0,34,18,L5 PT CTX,Cortical Neurons V
4,5,C3m1,6.0,6,15,Oligio,Oligo II
5,6,C3m1,16.0,16,2,Astro,Astro II
6,7,C3m1,30.0,30,2,Astro,Astro II
...,...,...,...,...,...,...,...
556312,62685,R61,159.0,159,1,Astro,Astro I
556313,62686,R61,97.0,97,0,Endo,Endo
556314,62687,R61,86.0,86,5,,Striatal Neurons I
556315,62688,R61,5.0,5,2,Astro,Astro II


In [12]:
xenium_overlap_scdata.obs

,cell_id,Sample ID,Number of genes,Number of transcripts,leiden,cluster_names,celltype
1,2,C3m1,12.0,12,12,Micro-PVM,Microglia
2,3,C3m1,34.0,34,18,L5 PT CTX,Cortical Neurons V
4,5,C3m1,6.0,6,15,Oligio,Oligo II
5,6,C3m1,16.0,16,2,Astro,Astro II
6,7,C3m1,30.0,30,2,Astro,Astro II
...,...,...,...,...,...,...,...
556312,62685,R61,159.0,159,1,Astro,Astro I
556313,62686,R61,97.0,97,0,Endo,Endo
556314,62687,R61,86.0,86,5,,Striatal Neurons I
556315,62688,R61,5.0,5,2,Astro,Astro II


In [13]:
xenium_overlap_scdata.obs['Number of transcripts'] = list(xenium_overlap_scdata.X.sum(axis=1))

C:\Users\krist\AppData\Local\Temp\ipykernel_27864\3820177663.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  xenium_overlap_scdata.obs['Number of transcripts'] = list(xenium_overlap_scdata.X.sum(axis=1))


In [14]:
xenium_overlap_scdata.obs['Number of transcripts'].value_counts()

3.0      29837
4.0      29474
5.0      28755
2.0      28294
6.0      27394
         ...  
202.0        1
188.0        1
197.0        1
151.0        1
177.0        1
Name: Number of transcripts, Length: 176, dtype: int64

In [15]:
xenium_overlap_scdata.obs['Number of genes'] = list(np.count_nonzero(xenium_overlap_scdata.X, axis = 1))

In [16]:
xenium_overlap_scdata.obs['Number of genes'].value_counts()

3     121562
4     110667
2      99806
5      77683
1      51985
6      42659
7      18939
0      15102
8       6458
9       1829
10       398
11        49
12         2
Name: Number of genes, dtype: int64

In [17]:
#set minimum number of gene threshold to 1
xenium_overlap_scdata_filtered = xenium_overlap_scdata[xenium_overlap_scdata.obs['Number of genes'] > 0]
xenium_overlap_scdata_filtered

View of AnnData object with n_obs × n_vars = 532037 × 13
    obs: 'cell_id', 'Sample ID', 'Number of genes', 'Number of transcripts', 'leiden', 'cluster_names', 'celltype'
    uns: 'Sample ID_colors', 'cell_id_colors', 'celltype_colors', 'cluster_names_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_spatial_rescaled', 'X_spatial_rescaled_flipped', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [18]:
#set minimum number of transcript threshold to 5
xenium_overlap_scdata_filtered = xenium_overlap_scdata_filtered[xenium_overlap_scdata_filtered.obs['Number of transcripts'] >= 5]
xenium_overlap_scdata_filtered

View of AnnData object with n_obs × n_vars = 420806 × 13
    obs: 'cell_id', 'Sample ID', 'Number of genes', 'Number of transcripts', 'leiden', 'cluster_names', 'celltype'
    uns: 'Sample ID_colors', 'cell_id_colors', 'celltype_colors', 'cluster_names_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_raw', 'X_spatial', 'X_spatial_rescaled', 'X_spatial_rescaled_flipped', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [19]:
#subset to include only 2 C24 and 2 HD24
sample_id_subset = ['C24m1', 'C24m2', 'HD24m1', 'HD24m2']
xenium_overlap_scdata_filtered = xenium_overlap_scdata_filtered[np.isin(xenium_overlap_scdata_filtered.obs['Sample ID'], sample_id_subset)]
xenium_overlap_scdata_filtered

KeyboardInterrupt: 

### Cosmx preprocessing

In [ ]:
cosmx_overlap_scdata = cosmx_scdata
cosmx_overlap_scdata.X = cosmx_overlap_scdata.obsm['X_raw']
cosmx_overlap_scdata = cosmx_scdata[:, var_names]
cosmx_overlap_scdata

In [ ]:
cosmx_scdata.obs

In [ ]:
cosmx_overlap_scdata.obs

In [ ]:
cosmx_overlap_scdata.obs['total_counts'] = list(cosmx_overlap_scdata.X.sum(axis = 1))

In [ ]:
cosmx_overlap_scdata.obs['total_counts'].value_counts()

In [ ]:
cosmx_overlap_scdata.obs['gene_count'] = list(np.count_nonzero(cosmx_overlap_scdata.X, axis = 1))

In [ ]:
cosmx_overlap_scdata.obs['gene_count'].value_counts()

In [ ]:
cosmx_overlap_scdata_filtered = cosmx_overlap_scdata[cosmx_overlap_scdata.obs['gene_count'] > 0]
cosmx_overlap_scdata_filtered

In [ ]:
cosmx_overlap_scdata_filtered = cosmx_overlap_scdata_filtered[cosmx_overlap_scdata_filtered.obs['total_counts'] >= 5]
cosmx_overlap_scdata_filtered

### Gellyfish preprocessing

In [ ]:
gellyfish_overlap_scdata = gellyfish_scdata
gellyfish_overlap_scdata.X = gellyfish_overlap_scdata.layers['Raw counts']
gellyfish_overlap_scdata = gellyfish_overlap_scdata[:, var_names]
gellyfish_overlap_scdata

In [ ]:
gellyfish_scdata.obs

In [ ]:
gellyfish_overlap_scdata.obs

In [ ]:
gellyfish_overlap_scdata.obs['total_counts'] = list(gellyfish_overlap_scdata.X.sum(axis = 1))                                       

In [ ]:
gellyfish_overlap_scdata.obs['n_genes'] = list(np.count_nonzero(gellyfish_overlap_scdata.X, axis = 1))

In [ ]:
gellyfish_overlap_scdata_filtered = gellyfish_overlap_scdata[gellyfish_overlap_scdata.obs['n_genes'] > 0]
gellyfish_overlap_scdata_filtered

In [ ]:
gellyfish_overlap_scdata_filtered = gellyfish_overlap_scdata_filtered[gellyfish_overlap_scdata_filtered.obs['total_counts'] >= 5]
gellyfish_overlap_scdata_filtered

### Xenium Overlap

In [ ]:
dimred = PCA()
dimred.fit(xenium_overlap_scdata_filtered.X)
sns.lineplot(np.cumsum(dimred.explained_variance_ratio_))
location = KneeLocator(range(1, len(dimred.explained_variance_ratio_) + 1),
                       np.cumsum(dimred.explained_variance_ratio_),
                       curve="concave", direction="increasing")
plt.axvline(location.elbow, color="red", linestyle="--")
plt.xlim((0, 5));

In [ ]:
sc.pp.pca(xenium_overlap_scdata_filtered, n_comps = 4)

In [ ]:
umap_operator = umap.UMAP(n_components = 2, random_state = 42, metric = 'euclidean', min_dist = 0.01, n_neighbors = 50)
umap_result_xenium = umap_operator.fit_transform(xenium_overlap_scdata_filtered.X)
xenium_overlap_scdata_filtered.obsm['X_umap'] = umap_result_xenium

In [ ]:
sc.pp.neighbors(xenium_overlap_scdata_filtered, use_rep = 'X_umap', n_neighbors=15)

In [ ]:
sc.tl.leiden(xenium_overlap_scdata_filtered, resolution = 0.001)

In [ ]:
sc.pl.umap(xenium_overlap_scdata_filtered, color = 'leiden', legend_loc = 'on data')

In [ ]:
xenium_overlap_scdata_filtered

In [ ]:
sc.pl.embedding(xenium_overlap_scdata_filtered, basis= 'X_spatial_rescaled_flipped', color = 'leiden', size = 5)

In [ ]:
adata_xenium_overlap = Jenie(xenium_overlap_scdata_filtered, 'xenium_overlap_scdata_filtered', 'xenium_overlap_scdata_filtered', pallete = 'color_dict')

In [ ]:
adata_xenium_overlap.umap(group='leiden', size = 3)

In [ ]:
#Note: ask Roy if I should filter out any clusters
xenium_overlap_scdata_filtered.obs['leiden'].value_counts()

### Cosmx Overlap

In [ ]:
dimred = PCA()
dimred.fit(cosmx_overlap_scdata_filtered.X)
sns.lineplot(np.cumsum(dimred.explained_variance_ratio_))
location = KneeLocator(range(1, len(dimred.explained_variance_ratio_) + 1),
                       np.cumsum(dimred.explained_variance_ratio_),
                       curve="concave", direction="increasing")
plt.axvline(location.elbow, color="red", linestyle="--")
plt.xlim((0, 15));

In [ ]:
#sc.pp.log1p(cosmx_overlap_scdata_filtered)
#sc.pp.normalize_total(cosmx_overlap_scdata_filtered)
#sc.pp.scale(cosmx_overlap_scdata_filtered)
#sc.pp.pca(cosmx_overlap_scdata_filtered, n_comps = 5, random_state = 42)
umap_operator = umap.UMAP(n_components = 2, random_state = 42, metric = 'euclidean', min_dist = 0.01, n_neighbors = 40)
umap_result_cosmx = umap_operator.fit_transform(cosmx_overlap_scdata_filtered.X)
cosmx_overlap_scdata_filtered.obsm['X_umap'] = umap_result_cosmx

In [ ]:
sc.pp.neighbors(cosmx_overlap_scdata_filtered, use_rep = 'X_umap', n_neighbors = 40)

In [ ]:
sc.tl.leiden(cosmx_overlap_scdata_filtered, resolution = 0.001)

In [ ]:
sc.pl.umap(cosmx_overlap_scdata_filtered, color = 'leiden', legend_loc = 'on data')

In [ ]:
print(cosmx_overlap_scdata_filtered.obs['leiden'].value_counts()[0:60])

In [ ]:
sc.pl.embedding(cosmx_overlap_scdata_filtered, basis = 'X_spatial', color = 'leiden')

In [ ]:
adata_cosmx_overlap = Jenie(cosmx_overlap_scdata_filtered, 'cosmx_overlap_scdata_filtered', 'cosmx_overlap_scdata_filtered', pallete = 'color_dict')

In [ ]:
adata_cosmx_overlap.umap(group = 'leiden', size=3)

### Gellyfish Overlap

In [ ]:
sc.pp.pca(gellyfish_overlap_scdata_filtered)
sc.pp.neighbors(gellyfish_overlap_scdata_filtered)
sc.tl.umap(gellyfish_overlap_scdata_filtered)

In [ ]:
sc.tl.leiden(gellyfish_overlap_scdata_filtered, resolution = 1)

In [ ]:
sc.pl.umap(gellyfish_overlap_scdata_filtered, color = 'leiden', legend_loc = 'on data')

In [ ]:
gellyfish_overlap_scdata_filtered.obs['leiden'].value_counts()